<a href="https://colab.research.google.com/github/chuancao26/islp/blob/main/LaboratorioResampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ISLP

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.5/849.5 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 5.6 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=1f66222ec5c4de457fdfeef6be035849a0c85e4a228d001656e536c3dd60b30b
  Stored in directory: /root/.cache/pip/wheels/50/37/21/0a719b9d89c635e89ff24bd93b862882ad675279552013b2fb
Successfully built autograd-gamma


In [2]:
# Librerias necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from functools import partial
import statsmodels.api as sm

from ISLP import load_data
from ISLP.models import sklearn_sm
from ISLP.models import (ModelSpec as MS,
                         summarize,
                         poly)

from sklearn.model_selection import train_test_split
from sklearn.base import clone
from sklearn.model_selection import (cross_validate,
                                     KFold,
                                     ShuffleSplit)





# Validation set


In [3]:
# train test split
auto = load_data("Auto")
auto_train, auto_test = train_test_split(auto,
                                         test_size=0.5,
                                         random_state=0)

In [ ]:
# fitting a ols model
design = MS(["horsepower"])
X_train = design.fit_transform(auto_train)
y_train = auto_train["mpg"]

model = sm.OLS(y_train, X_train)
results = model.fit()

X_test = design.fit_transform(auto_test)
predicted = results.predict(X_test)
np.mean((auto_test["mpg"] - predicted)**2)


np.float64(23.61661706966988)

In [5]:
def eval_MSE(terms,
             response,
             train,
             test):
  design = MS(terms)
  X_train = design.fit_transform(train)
  y_train = train[response]

  X_test = design.transform(test)
  y_test = test[response]

  model = sm.OLS(y_train, X_train)
  results = model.fit()
  predicted = results.predict(X_test)

  return np.mean((y_test - predicted)**2)


In [6]:
terms = [poly("horsepower", 3)]
print(eval_MSE(terms, "mpg",
               auto_train,
               auto_test))

16.974378328028724


In [7]:
# veamos con diferentes grados
predicted_mse = [eval_MSE([poly("horsepower", i + 1)],
                              "mpg",
                              auto_train,
                              auto_test) for i in range(3)]

In [8]:
predicted_mse

[np.float64(20.75540795922861),
 np.float64(16.94510675951611),
 np.float64(16.974378328028724)]

In [9]:
# usando otro se=ed
auto_train, auto_test = train_test_split(auto,
                                         test_size=0.5,
                                         random_state=3)

# veamos con diferentes grados
predicted_mse = [eval_MSE([poly("horsepower", i + 1)],
                              "mpg",
                              auto_train,
                              auto_test) for i in range(3)]
print(predicted_mse)

[np.float64(20.75540795922861), np.float64(16.94510675951611), np.float64(16.974378328028724)]


## Cross Validation

In [12]:
hp_model = sklearn_sm(sm.OLS,
                      MS(["horsepower"]))
X, y = auto.drop(["mpg"] ,axis=1), auto["mpg"]
cv_results = cross_validate(hp_model,
                            X,
                            y,
                            cv = auto.shape[0])
cv_err = np.mean(cv_results["test_score"])
cv_err


np.float64(24.23151351792922)